In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.manifold import TSNE

from tensorflow.keras import layers
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from highlight_text import ax_text, fig_text

import altair as alt

In [2]:
data = pd.read_csv('./combined_sentiment_labelled.tsv', sep='\t')

In [3]:
# data

In [4]:
# len(set([x for l in data.text.apply(lambda x: x.split(" ")).values for x in l]))

https://www.tensorflow.org/tutorials/text/text_classification_rnn

In [5]:
dataset = tfds.load('imdb_reviews',
                    as_supervised=True)

In [6]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [7]:
# for example, label in train_dataset.take(1):
#   print('text: ', example.numpy())
#   print('label: ', label.numpy())

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary())+2,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=['accuracy'])

In [11]:
# with tf.device('/GPU:0'):
#     model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#                   optimizer=tf.keras.optimizers.Adam(1e-4),
#                   metrics=['accuracy'])
#     history = model.fit(train_dataset, epochs=10,
#                         validation_data=test_dataset, 
#                         validation_steps=30)

In [12]:
# model.save_weights('first_model/')

In [13]:
model.load_weights('first_model/')

----

In [14]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    GREEN = '\033[92m'
    WARNING = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [15]:
text = data.text[7]


def color_text(text, model=model):
    tokens = text.split(" ")
    probs = [0]
    for k in range(0,len(tokens)):
        probs.append(model.predict(np.array([" ".join(tokens[:k+1])]))[0][0])
    pred = probs[-1]
    probs = np.diff(probs)
    colors = [bcolors.ENDC if abs(p / max(np.abs(probs))) < 0.1 
                  else (bcolors.RED if p < 0 else bcolors.GREEN) 
              for p in probs]
    ends = [bcolors.ENDC] * len(probs)
    return " ".join([c+t+e for c,t,e in zip(colors, tokens, ends)]), pred

https://stackoverflow.com/questions/287871/how-to-print-colored-text-in-python

In [16]:
for text in data.text[:20]:
    pred = color_text(text)
    print("NEGATIVE" if pred[1] < 0 else "POSITIVE", "|", pred[0])

NEGATIVE | So there is no way for me to plug it in here in the US unless I go by a converter.
POSITIVE | Good case, Excellent value.
POSITIVE | Great for the jawbone.
POSITIVE | Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
POSITIVE | The mic is great.
NEGATIVE | I have to jiggle the plug to get it to line up right to get decent volume.
POSITIVE | If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.
NEGATIVE | If you are Razr owner...you must have this!
NEGATIVE | Needless to say, I wasted my money.
NEGATIVE | What a waste of money and time!.
POSITIVE | And the sound quality is great.
NEGATIVE | He was very impressed when going from the original battery to the extended battery.
NEGATIVE | If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset.
POSITIVE | Very good quality though
POSITIVE | The design is very odd, as the ear "clip" is no

-----------

### Average of word embeddings

In [17]:
def sen2vec(x):
    return model.get_layer(name='embedding')(model.get_layer(name="text_vectorization")(x))

In [18]:
a = sen2vec([[x] for x in data.text.values[:3]])

In [19]:
# samples, words, embedding
a.shape

TensorShape([3, 21, 64])

### Processed sentences

In [20]:
sen2vec_model = tf.keras.Sequential([
    model.get_layer(name="text_vectorization"),
    model.get_layer(name='embedding'),
    model.get_layer(name='lstm'),
    model.get_layer(name='dense')
])

In [21]:
sentences = data.sample(n=50).text.values

tsne = TSNE()
tsned_space_raw = tsne.fit_transform(sen2vec([[x] for x in sentences]).numpy().mean(axis=1))

tsned_space_proc = tsne.fit_transform(sen2vec_model.predict(sentences))

In [27]:
tsne_plot_data = pd.DataFrame({'x_raw': tsned_space_raw[:,0], 
                               'y_raw': tsned_space_raw[:,1],
                               'x_proc': tsned_space_proc[:,0], 
                               'y_proc': tsned_space_proc[:,1],
                               'sentence': sentences, 
                               'prob': model.predict(sentences).reshape(-1).round(2).astype(str),
                               'pred': ['Positive' if x else 'Negative' 
                                        for x in (model.predict(sentences).reshape(-1) > 0)]})

In [28]:
selector_raw = alt.selection_interval(empty='none', encodings=['x', 'y'])
selector_proc = alt.selection_interval(empty='none', encodings=['x', 'y'])

In [29]:
words_tsned = alt.Chart(tsne_plot_data).mark_circle(size=200).encode(
    x = 'x_raw',
    y = 'y_raw',
    tooltip =[alt.Tooltip('sentence'), alt.Tooltip('prob')],
    color = alt.Color('pred', scale=alt.Scale(domain=['Negative', 'Positive'], 
                                              range=['red', 'green'])),
    opacity=alt.condition(selector_proc, alt.value(1), alt.value(0.1))
).properties(
    title='Raw sentences'
).add_selection(
    selector_raw
)

In [30]:
sentences_tsned = alt.Chart(tsne_plot_data).mark_circle(size=200).encode(
    x = 'x_proc',
    y = 'y_proc',
    tooltip =[alt.Tooltip('sentence'), alt.Tooltip('prob')],
    color = alt.Color('pred', scale=alt.Scale(domain=['Negative', 'Positive'], 
                                              range=['red', 'green'])),
    opacity=alt.condition(selector_raw, alt.value(1), alt.value(0.1))
).properties(
    title='Processed sentences'
).add_selection(
    selector_proc
)

In [31]:
words_tsned | sentences_tsned

alt.HConcatChart(...)